In [1]:
import pandas as pd
import librosa
import os

In [5]:
def read_fma_audio(path):
    audio_files= []
    for subdir, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".mp3") or file.endswith(".wav"):
                file_path= os.path.join(subdir, file)
                # read audio file with no sampling rate
                data, sampling_rate= librosa.load(file_path, sr= None)  
                duration= librosa.get_duration(y= data, sr= sampling_rate)
                # append information to the list
                audio_files.append({
                    'file_path': file_path,
                    'duration': duration,
                    'format': file.split('.')[-1]
                })
    return audio_files

In [6]:
dataset_path= 'noob_sample'
audios= read_fma_audio(dataset_path)

In [7]:
audio_df= pd.DataFrame(audios)
audio_df.head()

,file_path,duration,format
0,noob_sample/000/000498.mp3,29.976576,mp3
1,noob_sample/000/000549.mp3,29.976576,mp3
2,noob_sample/000/000568.mp3,29.976576,mp3
3,noob_sample/000/000517.mp3,30.002698,mp3
4,noob_sample/000/000540.mp3,29.976576,mp3


## MetaData ##

/tmp/ipykernel_18878/3591649982.py:2: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  tracks = pd.read_csv('fma_metadata/tracks.csv')
